In [5]:
!pip install --upgrade pip
!pip install spacy

Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 1.7MB 707kB/s eta 0:00:01
  Found existing installation: pip 9.0.1
    Uninstalling pip-9.0.1:
Exception:
Traceback (most recent call last):
  File "/public/apps/anaconda3/5.0.1/lib/python3.6/shutil.py", line 544, in move
    os.rename(src, real_dst)
OSError: [Errno 18] Invalid cross-device link: '/public/apps/anaconda3/5.0.1/bin/pip' -> '/tmp/pip-vd3xky3o-uninstall/public/apps/anaconda3/5.0.1/bin/pip'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/public/apps/anaconda3/5.0.1/lib/python3.6/site-packages/pip/basecommand.py", line 215, in main
    status = self.run(options, args)
  File "/public/apps/anaconda3/5.0.1/lib/python3.6/site-packages/pip/commands/install.py", line 342, in run
    prefix=options.prefix_path,
  File "/public/apps/anaconda3/5.0.1/lib/python3.6/site-packages/pip/req/req_set.py", line 778, in install
    req

Command "python setup.py egg_info" failed with error code 1 in /tmp/pip-build-j9a5clx6/spacy/
You are using pip version 9.0.1, however version 21.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [101]:
from typing import Any, List, Tuple
import numpy as np
import spacy
from spacy.lang.en import English
from spacy.tokenizer import Tokenizer
import copy
import random
import tqdm
import os
import glob
from pathlib import Path

import torch

# Initial seed
# random.seed(42)


class CustomToken:
    def __init__(self, token, whitespace=""):
        self.token = token
        self.whitespace = whitespace

    def get_text(self):
        token = copy.deepcopy(self.token)
        token = token + self.whitespace
        return token

    def __str__(self):
        return self.token


def has_pre_space(token):
    if token.i == 0:
        return False
    if token.nbor(-1).whitespace_:
        return True
    else:
        return False


def has_space(token):
    if type(token) == spacy.tokens.token.Token:
        return token.whitespace_
    elif type(token) == CustomToken:
        return token.whitespace


def revtok(tokens):
    retok = "".join([str(tokens[i]) + has_space(tokens[i]) for i in range(len(tokens))])
    # assert retok == str(tokens)
    return retok


def derandomize(sent, pos, tokenizer, separator=""):
    toks = [str(s) for s in tokenizer(sent)]
    return separator.join(toks[i] for i in np.argsort(pos))


def fixed_shuffle_numpy(lst, mask, percent=1.0, allow_kept_order=False):
    to_permute = [i for i, m in enumerate(mask) if m]
    num_permute = int(len(to_permute) * percent)
    num_to_fix = len(to_permute) - num_permute
    if num_to_fix > 0:
        to_fix_ids = random.sample(to_permute, num_to_fix)
        mask = [True if mask[i] or i in to_fix_ids else False for i in range(len(mask))]
    # mask = [True if mask[i] or random.uniform(0,1) > percent else False for i,e in enumerate(mask)]
    unfrozen_indices = [i for i, e in enumerate(lst) if not mask[i]]
    unfrozen_set = lst[unfrozen_indices]
    unfrozen_set_og = copy.deepcopy(unfrozen_set)
    if len(unfrozen_set) > 1:
        while True:
            random.shuffle(unfrozen_set)
            same_pos = sum(
                [
                    1
                    for i, e in enumerate(unfrozen_set)
                    if unfrozen_set[i] == unfrozen_set_og[i]
                ]
            )
            if same_pos == 0 or allow_kept_order:
                break
    lst[unfrozen_indices] = unfrozen_set


def get_available_start_positions(tokens, num_to_freeze):
    """Return all possible start positions by keeping into
    account the CustomToken

    Args:
        tokens ([type]): [description]
        num_to_freeze ([type]): [description]
    """
    pos = []
    for i in range(len(tokens) - num_to_freeze):
        if not has_custom_token(tokens[i : i + num_to_freeze]):
            pos.append(i)
    return pos


def freeze_contiguous_chunk(tokens, num_to_freeze) -> Tuple[List[Any], bool]:
    """Freeze contigious chunks of the sentence
    Randomly select the start position to freeze

    Args:
        tokens ([type]): sentence in list of words
        num_to_freeze ([type]): number of words to freeze

    Returns:
        [type]: sentence and boolean flag whether the any parts are chunked
    """
    # tok = copy.deepcopy(tokens)
    tok = tokens
    available_start_positions = get_available_start_positions(tokens, num_to_freeze)
    if len(available_start_positions) == 0:
        return tok, False
    start_id_to_keep = random.choice(available_start_positions)
    end_id_to_keep = start_id_to_keep + num_to_freeze
    new_tok = []
    build = []
    for ti in range(len(tok)):
        if ti == end_id_to_keep:
            new_tok.append(CustomToken(revtok(build), whitespace=build[-1].whitespace_))
        if ti >= start_id_to_keep and ti < end_id_to_keep:
            build.append(tok[ti])
        else:
            new_tok.append(tok[ti])
    return new_tok, True


def has_custom_token(tokens):
    for tok in tokens:
        if type(tok) == CustomToken:
            return True
    return False


def split_by_custom_token(tokens):
    token_pos = -1
    for i, tok in enumerate(tokens):
        if type(tok) == CustomToken:
            token_pos = i
            break
    return tokens[:token_pos], tokens[token_pos + 1 :]


def freeze_ngrams(tokens, n=3):
    """Freeze n-grams in the given sentence


    Args:
        tok ([type]): [description]
        n (int, optional): [description]. Defaults to 3.

    Returns:
        [type]: [description]
    """

    while True:
        tokens, changed = freeze_contiguous_chunk(tokens, n)
        if not changed:
            break

    return tokens


def freeze_noun_chunks(tokens):
    """
    Given a sentence, freeze the noun chunks in a CustomToken
    Args:
        tokens ([type]): Spacy Doc object
    """
    chunk_boundaries = [(chunk.start, chunk.end) for chunk in tokens.noun_chunks]
    in_chunk = np.array([False] * len(tokens))
    for cb in chunk_boundaries:
        in_chunk[cb[0] : cb[1]] = True
    new_tokens = []
    build = []
    for ti, token in enumerate(tokens):
        if in_chunk[ti]:
            build.append(token)
        else:
            if len(build) > 0:
                new_tokens.append(
                    CustomToken(revtok(build), whitespace=build[-1].whitespace_)
                )
                build = []
            new_tokens.append(token)
    if len(build) > 0:
        new_tokens.append(CustomToken(revtok(build), whitespace=build[-1].whitespace_))
    return new_tokens


def randomize(
    sent,
    retain_stop=False,
    retain_punct=False,
    keep_order=0.0,
    num_to_keep=0,
    keep_n_grams=1,
    keep_noun_chunks=False,
    percent=1.0,
    seed=42,
    lang="en",
    tokenizer=None,
    allow_kept_order=False
):
    """Randomize a given sentence such that no word is in the original position

    Args:
        sent ([type]): Input sentence
        retain_stop (bool, optional): Retain stopwords to its original location. Defaults to False.
        retain_punct (bool, optional): Retain punctuations in its original location. Defaults to False.
        keep_order (float, optional): Percentage of words to keep together. num_to_keep supersedes this. Defaults to 0.0.
        num_to_keep (int, optional): Number of words to keep together. Defaults to 0.
        keep_n_grams (int, optional): n-grams to keep together in the permutation. Defaults to 1.
        keep_noun_chunks (bool, optional): freeze the noun chunks together, defaults to False
        percent (float, optional): Additional flag to control percentage of shuffled tokens. Defaults to 1.0.
        seed (int, optional): Seed for reproducibility. Defaults to 42.
        lang (str, optional): English (you can add more languages here). Defaults to "en".
        tokenizer ([type], optional): Spacy tokenizer. Defaults to None.

    Raises:
        AssertionError: Raises error if num_to_keep = len(tokens)

    Returns:
        new_sent (str): permuted sentence
        indx (list[int]): indexes of the original tokens
        ch (float): percentage of words changed from its original position
        mask (list[boolean]): masks used to fix word position
        kept_order (boolean): True if any words is kept in its original position
    """
    #random.seed(seed)
    tok = tokenizer(sent)
    kept_order = False
    if keep_noun_chunks:
        tok = freeze_noun_chunks(tok)
    if num_to_keep == 0:
        num_to_keep = int(len(tok) * keep_order)
    if num_to_keep > 0:
        if num_to_keep == len(tok):
            raise AssertionError("Cannot shuffle if all words are kept")
        tok, _ = freeze_contiguous_chunk(tok)
        kept_order = True
    if keep_n_grams > 1:
        tok = freeze_ngrams(tok, n=keep_n_grams)
    masks = []
    masks.append(np.zeros(len(tok)).astype(bool))
    if retain_punct:
        masks.append(
            np.array(
                [
                    t.is_punct if type(t) == spacy.tokens.token.Token else False
                    for t in tok
                ]
            )
        )
    if retain_stop:
        masks.append(
            np.array(
                [
                    t.is_stop if type(t) == spacy.tokens.token.Token else False
                    for t in tok
                ]
            )
        )
    mask = np.any(np.stack(masks), axis=0)
    indx = np.arange(len(tok))
    fixed_shuffle_numpy(indx, mask, percent, allow_kept_order)
    new_tok = [tok[i] for i in indx]
    if lang == "en":
        separator = " "
    else:
        separator = ""
    new_sent = separator.join([str(t) for t in new_tok])
    if lang == "en":
        # removing extra space
        new_sent = separator.join(new_sent.split())
    # not possible for chinese as the tokenization words matter on randomization
    # de_rand = derandomize(new_sent, indx, tokenizer)
    # assert de_rand == sent
    # percent changed
    ch = sum([1 for i, p in enumerate(indx) if i != p and not mask[i]]) / sum(
        [1 for m in mask if not m]
    )
    return new_sent, indx, ch, mask, kept_order

In [98]:
seed = 1
random.seed(seed)
np.random.seed(seed)
sent = "It is situated on the river Ilm, southeast of Ingolstadt ."
nlp = English()

sent, indx, ch, mask, kept_order = randomize(sent, 
                                             seed=1, 
                                             keep_n_grams=1, 
                                             retain_punct=True, 
                                             tokenizer=nlp,
                                             allow_kept_order=True)
print(sent)

unfrozen_set:[ 0  1  2  3  4  5  6  8  9 10]
unfrozen_set_og:[ 0  1  2  3  4  5  6  8  9 10]
Ilm of Ingolstadt southeast river on It , the is situated .


In [96]:
indx

array([ 3, 10,  4,  0,  2,  5,  8,  7,  1,  9,  6, 11])

In [73]:
ch

1.0

In [74]:
mask

array([False, False, False, False, False, False, False,  True, False,
       False, False,  True])

In [75]:
kept_order

False

In [99]:
def unbpe(line):
    line = line.strip().split(' ')
    new_line = ""
    for token in line:
        if token.endswith('@@'): # starting subword
            new_line+= token[:-2]
        elif token.startswith('@@'): # endind subword
            new_line = new_line[:-1]
            new_line+= token[2:]
            new_line+= ' '
        else: # whole word token
            new_line+= token + ' '
    
    return new_line[:-1]

def create_permuted_data(org_fname, dest_fname, seed, keep_n_grams):
    random.seed(seed)
    np.random.seed(seed)
    f_out = open(dest_fname, 'w+')
    with open(org_fname, 'r') as f_input:
        for line in f_input.readlines():
            line = unbpe(line)
            #print(f"line:{line}")
            sent, indx, ch, mask, kept_order = randomize(line.strip(), 
                                                 seed=1, 
                                                 keep_n_grams=5, 
                                                 retain_punct=True, 
                                                 tokenizer=nlp,
                                                 allow_kept_order=True)
            #print(f"sent:{sent}")
            f_out.write(sent+'\n')
    f_out.close()
    

In [38]:
# permute syntatic
org_file = '/private/home/dieuwkehupkes/nmt/compositional_mt/systematicity/s_conj/synthetic/systematicity_synthetic_1_s1_s2.en'
dest_file = '/checkpoint/itayitzhak/permuted_data/systematicity/s_conj/synthetic/systematicity_synthetic_1_s1_s2_5gram.en'
create_permuted_data(org_file, dest_file, seed=1)


line:The father definitely avoids the director , and the boys criticise the leader .

sent:definitely avoids the director , the boys criticise the leader and The father .
line:The butchers that cycle recognise the queen , and the athletes criticise the director .

sent:and the athletes criticise the that cycle recognise the queen director , butchers The .
line:The girls behind the car recognise the president , and the man forgets the queen .

sent:man the forgets queen the president , and the The girls behind the car recognise .
line:The victim beside the train admires the general , and the athletes observe the king .

sent:the observe athletes the general , and the king admires The victim beside the train .
line:The mothers probably want that the poets scream , and the woman understands the leader .

sent:scream leader and the woman understands the The , mothers probably want that the poets .
line:The team of boys near the train observes the general , and the athletes understand the g

sent:uncles consciously see that the poets sleep , and the the director The teachers avoid .
line:The athletes behind the bike understand the leader , and the girls admire the director .

sent:the leader director , and the girls admire athletes behind the bike understand The the .
line:The children see that the uncles scream , and the woman criticises the king .

sent:king The and the woman criticises the , children see that the uncles scream .
line:The team of farmers behind the truck observes the general , and the baker recognises the director .

sent:and The team of farmers behind the the baker recognises the director truck observes the general , .
line:The poet behind the bike avoids the president , and the boy criticises the president .

sent:the the president , and the boy criticises The president poet behind the bike avoids .
line:The farmers and the uncles observe the general , and the farmer criticises the mayor .

sent:mayor The farmers and the uncles observe the the , and th

In [58]:
# permute OPUS small train
org_file = '/checkpoint/dieuwkehupkes/mt-data/en-nl/opus-taboeta/train.tok.shuf.small.bpe.60000.en'
dest_file = '/checkpoint/itayitzhak/permuted_data/mt-data/en-nl/opus-taboeta/train.tok.shuf.small.en'
create_permuted_data(org_file, dest_file, seed=1, keep_n_grams=1)

In [69]:
# permute OPUS test
org_file = '/checkpoint/dieuwkehupkes/mt-data/en-nl/opus-taboeta/test.tok.bpe.60000.en'
dest_file = '/checkpoint/itayitzhak/permuted_data/mt-data/en-nl/opus-taboeta/test.tok.en'
create_permuted_data(org_file, dest_file, seed=1, keep_n_grams=1)

In [70]:
# permute OPUS valid
org_file = '/checkpoint/dieuwkehupkes/mt-data/en-nl/opus-taboeta/dev.tok.bpe.60000.en'
dest_file = '/checkpoint/itayitzhak/permuted_data/mt-data/en-nl/opus-taboeta/valid.tok.en'
create_permuted_data(org_file, dest_file, seed=1, keep_n_grams=1)

In [105]:
# permute OPUS all train
#for seed in [1,2,3,4,5]:
org_file = '/checkpoint/dieuwkehupkes/mt-data/en-nl/opus-taboeta/train.tok.shuf.bpe.60000.en'
dest_file = f'/checkpoint/itayitzhak/permuted_data/mt-data/en-nl/opus-taboeta/train.5gram.tok.shuf.all.en'
create_permuted_data(org_file, dest_file, seed=seed, keep_n_grams=5)


In [ ]:
# permute OPUS small train
for seed in [1,2,3,4,5]:
    org_file = '/checkpoint/dieuwkehupkes/mt-data/en-nl/opus-taboeta/train.tok.shufsmall.bpe.60000.en'
    dest_file = f'/checkpoint/itayitzhak/permuted_data/mt-data/en-nl/opus-taboeta/train.{seed}.tok.shufsmall.en'
    create_permuted_data(org_file, dest_file, seed=seed, keep_n_grams=1)

In [ ]:
cp /checkpoint/itayitzhak/permuted_data/mt-data/en-nl/opus-taboeta/test.tok.bpe.60000.nl /checkpoint/itayitzhak/permuted_data/mt-data/en-nl/opus-taboeta/test.5gram.tok.bpe.60000.nl